In [ ]:
import uproot
import awkward as ak
import numpy as np

# Open File

In [ ]:
file = uproot.open("/lstr/sahara/act/data/DAOD_PHYSLITE.37621365._000015.pool.root.1")

## Get to know your data
MC simulation electroweak boson nominal samples from the ATLAS experiment. 

### Exercise 1:

What objects are in this file? What's the name of the ttree?

In [ ]:
file.keys()

In [ ]:
# Access ttree
tree = file["insert_tree_name_here"] # fix me

In [ ]:
# we can see the list of branch names using tree.keys()
tree.keys()

In [ ]:
# or you can use tree.show() to see more information on the branches
tree.show()

### Exercise 2

How many branches are in this ttree?
To answer this, you may want to use `len(...)`. 

In [ ]:
x = len(...) # fix me
print("Number of branches:", x)

### Exercise 3

We can read TBranches as arrays using `tree.arrays(["insert_list_of_branch_names_here"])`. Fill in the blanks associated with Muons (we want charge, pt, eta, phi).

In [ ]:
Electrons = tree.arrays(["AnalysisElectronsAuxDyn.charge", "AnalysisElectronsAuxDyn.pt","AnalysisElectronsAuxDyn.eta", "AnalysisElectronsAuxDyn.phi","AnalysisElectronsAuxDyn.m"])
Muons = tree.arrays([...]) # fix me

### Exercise 4
How many events are in this tree?

In [ ]:
num_events = ... # fix me
print("Number of events:", num_events)

### Exercise 5

How many electrons are in the 80th event? What are their charges? How many muons are in the 34th event? What are their pt? 

In [ ]:
Electrons["AnalysisElectronsAuxDyn.charge"][79]

In [ ]:
Muons["..."][...] # fix me

To make our lives easier, [ak.zip](https://awkward-array.org/doc/main/reference/generated/ak.zip.html) combines arrays into a single structure as a collection of records. 

In [ ]:
e = ak.zip({
    "pt" : Electrons["AnalysisElectronsAuxDyn.pt"],
    "eta" : Electrons["AnalysisElectronsAuxDyn.eta"],
    "phi" : Electrons["AnalysisElectronsAuxDyn.phi"],
    "mass" : Electrons["AnalysisElectronsAuxDyn.m"],
    "charge" : Electrons["AnalysisElectronsAuxDyn.charge"]
})
mu = ak.zip({
    "pt" : Muons["AnalysisMuonsAuxDyn.pt"],
    "eta" : Muons["AnalysisMuonsAuxDyn.eta"],
    "phi" : Muons["AnalysisMuonsAuxDyn.phi"],
    "charge" : Muons["AnalysisMuonsAuxDyn.charge"]
})

What's the total number of electrons in the data set? What about muons? The function [ak.num](https://awkward-array.org/doc/main/reference/generated/ak.num.html) may help.

In [ ]:
num_e = ... # fix me
num_mu = ... # fix me
print("Total number of electrons: ", num_e)
print("Total number of muons: ", num_mu)

# Ready for analysis

Now we are going to calculate the tranverse momentum components and energy for electrons, using the above.  

In [ ]:
#  Momentum components
def Px(pt, phi):
    return pt*np.cos(phi)

def Py(pt,phi):
    return pt*np.sin(phi)

def Pz(pt,eta):
    return pt*np.sinh(eta)

def E(pt, eta, m):
    return np.sqrt(pt**2 * np.cosh(eta)**2 + m**2)

e_px = Px(e.pt,e.phi)
e_py = Py(e.pt,e.phi)
e_pz = Pz(e.pt,e.eta)
e_E = E(e.pt,e.eta,e.mass)

The invariant mass of two oppositely charged leptons is defined as:
\begin{equation}
m_0 = \sqrt{(\sum{E})^2-(\sum{p_x})^2-(\sum{p_y})^2-(\sum{p_z})^2}
\end{equation}


### Exercise 6

Create a function, using the functions above for energy and momentum components, that calculates the invariant mass in GeV. Calculate the invariant mass of the electron pair in the 63rd event.

In [ ]:
def calc_invm(p1, p2):
    sumpx = ... # fix me
    sumpy = ... # fix me
    sumpz = ... # fix me 
    sumE = ... # fix me
    
    invm = np.sqrt(...) # fix me
    return invm/1000.0 # in GeV

## Data Selection

As shown in Frank's workshop, we can mask awkward arrays to apply cuts for data selection. 

### Exercise 7

Make a selection of events with more than two electrons and with electron pt greater than 25 GeV [data set is in MeV].

This should leave you with a total of 2130 electrons.

In [ ]:
e_cut = (__insert_cut_here__)
cleaned_e = e[e_cut]
# check 
filtered_electrons = ... 
print("Number of electrons that passed selection:", filtered_electrons)

### Exercise 8

Of these events, we want to calculate the invariant mass of oppositely charged pairs. However, some events include 3 electrons perhaps with charges [1,-1,1], which in this case have 2 possible pairings. 

How many events contain more than 2 electrons? What are their indices?

The function [ak.where](https://awkward-array.org/doc/main/reference/generated/ak.where.html) and [ak.combinations](https://awkward-array.org/doc/main/reference/generated/ak.combinations.html) may come in handy.

In [ ]:
lengths = ak.num(cleaned_e)
indices = ak.where(lengths > 2)[0].tolist()
indices

In [ ]:
cleaned_e[15180]

In [ ]:
h = ak.combinations(cleaned_e, 2, axis = 1)
h[15180]

# Homework Assignment

## Problem 1
Continuing on with the workshop, the goal of this assignment is to plot the Z mass using the tools we've learned from the awkward library. Z decays into electron-positron or muon-antimuon. 
<div style="display: flex;">
    <img src="https://atlas.physicsmasterclasses.org/zpath_files/img/highslide/feynman/Z_ElectronPositron.png" style="height:300px; margin-right: 10px;">
    <img src="https://atlas.physicsmasterclasses.org/zpath_files/img/highslide/feynman/Z_MyonAntimyon.png" style="height:300px;">
</div>

1. Create a function `selection_calc(lepton)` that applies the selection we've made above, obtains oppositely charged lepton pairs, and calculates the invariant mass. We want events with oppositely charged lepton pairs with pt > 25 GeV. \
    __hint__: Re-use the functions already built in class. \
    __note__: The data does not contain Muon mass, so you can assume all muons have mass = 0 (you might need to adjust the function `calc_invm` for this) 
   
2. Plot the invariant mass of both electron and muon pairs on the same histogram. Find the peak of the histogram and return the Z mass.

## Problem 2

Now you're on your own! We will make selections used for top quark mass calculation, using ATLAS ttbar mc sample, `/lstr/sahara/act/data/DAOD_PHYSLITE.37620644._000012.pool.root.1`.

1. Make a selection of events that have atleast one lepton with pt > 30GeV and abs(eta) < 2.1 , and at least four jets. The jets should also have pt >25 GeV and abs(eta) < 2.4. How many events were selected? Plot distribution plots to check selection.
 
2. Now let's check jet quality. The btag discriminant is defined as :
   \begin{equation}
   D_{DL1} = \log(\frac{p_b}{f_c \cdot p_c + (1-f_c) \cdot p_u})
   \end{equation}
   where $p_b, p_c, p_u$ are the fields labeled `BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pb`, `BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pc`, `BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pu`, and $f_c = 0.018$. Calculate the Btag discriminant of the selected jets.

3. A jet is considered as b-tagged if the $D_{DL1}$ variable is above threshold. The threshold value for 77% efficiency is `2.456`. Make another selection of events that have at least two b-tagged jets. How many events were selected? Plot distribution plots to check selection.

__Hint__: [ak.any(...)](https://awkward-array.org/doc/main/reference/generated/ak.any.html) can help.

## Problem 3

Using the provided ttbar sample from CMS,`/lstr/sahara/act/data/TTto2L2Nu_NanoAODv12-130x_mcRun3_13p6TeV_powheg-pythia.root`, calculate the efficiency of the HLT_Mu8 trigger for events with 1 muon as a function of muon pT. At what muon pT does the trigger 'turn on'? Can you guess what the trigger name means? Do this again, but for events with 2 muons. Explain any similarities/differences as they relate to the trigger.You may use coffea or uproot as your entry point into the file. \
__Note:__ The efficiency of a trigger is defined as the ratio of the number of events passing the trigger (HLT_Mu8 = True) to all events. This is often done for individual bins of some quantity. Histogram objects from the Hist package can be divided.